In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper, col
import os
import utils
import pyspark
from pyspark.sql import SparkSession,DataFrame
import requests
import json 
from io import BytesIO
import pandas as pd
import os
import sys
import utils as utils


In [3]:
spark = utils.create_context()
db_name = "landing"
table_name = "turismo_Provincia"



df = utils.read_iceberg_table(spark,db_name,table_name)
df.show()

+----+---+-----------+----------------+--------------------+--------------------+--------+--------------+--------------+
| AÑO|MES|CCAA_ORIGEN|PROVINCIA_ORIGEN|        CCAA_DESTINO|   PROVINCIA_DESTINO|TURISTAS|PERNOCTACIONES|ESTANCIA_MEDIA|
+----+---+-----------+----------------+--------------------+--------------------+--------+--------------+--------------+
|2019|  7|  Andalucía|         Almería|           Andalucía|               Cádiz|  8910.0|       33536.0|           3.8|
|2019|  7|  Andalucía|         Almería|           Andalucía|             Córdoba|  3468.0|       11981.0|           3.5|
|2019|  7|  Andalucía|         Almería|           Andalucía|             Granada| 49626.0|      141865.0|           2.9|
|2019|  7|  Andalucía|         Almería|           Andalucía|              Huelva|  2801.0|       10920.0|           3.9|
|2019|  7|  Andalucía|         Almería|           Andalucía|                Jaén|  9247.0|       34155.0|           3.7|
|2019|  7|  Andalucía|         A

In [ ]:
#!/usr/bin/env python3
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper, col

# Pin Python for Spark workers & driver
os.environ["PYSPARK_PYTHON"]        = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

def main():
    # 1) Determine base POC folder (one level up from CWD)
    cwd = os.getcwd()  
    poc_root = os.path.abspath(os.path.join(cwd, ".."))

    # 2) Landing parquet actually lives under POC/landing/data/landing/turismo_Provincia
    landing_dir = os.path.join(poc_root, "landing", "data", "landing", "turismo_Provincia")
    print("→ Reading landing Parquet from", landing_dir)

    # 3) Start Spark session
    spark = (
        SparkSession.builder
          .appName("TrustedCSVLoad")
          .config("spark.master", "local[*]")
          .config("spark.driver.memory", "2g")
          .getOrCreate()
    )

    try:
        # 4) Read landing Parquet
        if not os.path.isdir(landing_dir):
            raise FileNotFoundError(f"Landing dir not found: {landing_dir}")
        df = spark.read.parquet(landing_dir)

        # 5) Clean & normalize
        df_clean = df.dropna()
        for c in ["CCAA_ORIGEN","PROVINCIA_ORIGEN","CCAA_DESTINO","PROVINCIA_DESTINO"]:
            df_clean = df_clean.withColumn(c, upper(col(c)))

        # 6) Write trusted CSV under POC/trusted/data/trusted/…
        tgt_dir = os.path.join(poc_root, "trusted", "data", "trusted", "turismo_Provincia_clean_csv")
        os.makedirs(tgt_dir, exist_ok=True)
        print("→ Writing trusted CSV to", tgt_dir)

        (
            df_clean
              .repartition(1)
              .write
              .mode("overwrite")
              .option("header", True)
              .csv(tgt_dir)
        )

        print("✅ Trusted zone load complete (CSV).")

    finally:
        spark.stop()

if __name__ == "__main__":
    main()


→ Reading landing Parquet from c:\Users\joaqu\OneDrive\Documents\AAmaster_UPC\TFM\TravelMind\POC\landing\data\landing\turismo_Provincia
→ Writing trusted CSV to c:\Users\joaqu\OneDrive\Documents\AAmaster_UPC\TFM\TravelMind\POC\trusted\data\trusted\turismo_Provincia_clean_csv
✅ Trusted zone load complete (CSV).


In [2]:
# 1) Re-use the same Iceberg-aware session for read & write
spark = utils.create_context()


# Read landing-zone Iceberg table
src_db, src_tbl = "landing", "turismo_Provincia"
print(f"→ Reading spark_catalog.{src_db}.{src_tbl}")
df = utils.read_iceberg_table(spark, src_db, src_tbl)
# 5) Clean & normalize
df_clean = df.dropna()
for c in ["CCAA_ORIGEN","PROVINCIA_ORIGEN","CCAA_DESTINO","PROVINCIA_DESTINO"]:
    df_clean = df_clean.withColumn(c, upper(col(c)))
# Write into exploitation zone
tgt_db, tgt_tbl = "exploitation", "turismo_Provincia"
print(f"→ Writing spark_catalog.{tgt_db}.{tgt_tbl}")
utils.overwrite_iceberg_table(spark, df_clean, tgt_db, tgt_tbl)

print("✅ Exploitation load complete.")


spark.stop()


→ Reading spark_catalog.landing.turismo_Provincia
→ Writing spark_catalog.exploitation.turismo_Provincia


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\joaqu\.virtualenvs\TravelMind--WcSvJCM\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\joaqu\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\joaqu\.virtualenvs\TravelMind--WcSvJCM\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\joaqu\.virtualenvs\TravelMind--WcSvJCM\Lib\site-packages\py4j\clien

Py4JError: An error occurred while calling o64.createOrReplace